# Getting Started

In this assignment you will be implementing at least 2 unique models and working with both the CIFAR10 and Imagenette datasets. The cell below defines a basic model based on the examples we reviewed in class. You should use this as a starting point to implement your own models.

You should have no problems running this notebook in Google Colab. If you choose to run it on your own machine, a yaml file is provided with the necessary dependencies. If you are using anaconda, you can create a new environment with the following command:

```bash
conda env create -f env.yml
```

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics


class BaselineModel(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Linear(64 * 64, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


The Imagenette dataset is a smaller subset of 10 easily classified classes from Imagenet. It is available to download from `torchvision`, as shown in the cell below. There are 3 different sizes of the images available. Feel free to use whichever version you prefer. It might make a difference in the performance of your model.

**Note: After downloading the Imagenette dataset, you will need to set `download=False` in the cell below to avoid errors.**

In [ ]:
from torchvision import transforms
from torchvision.datasets import Imagenette
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint


# Prepare the dataset
train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=False, transform=train_transforms)

# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)
val_dataset.dataset.transform = test_transforms

# Use DataLoader to load the dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=False, transform=test_transforms)

model = BaselineModel()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

100%|██████████| 99003388/99003388 [00:16<00:00, 6100889.65it/s]


Extracting data/imagenette/train/imagenette2-160.tgz to data/imagenette/train/


100%|██████████| 99003388/99003388 [00:19<00:00, 5067238.04it/s]


Extracting data/imagenette/test/imagenette2-160.tgz to data/imagenette/test/


In [6]:
# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback], max_epochs=3)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode
--------------------------------------------------------
0 | estimator | Sequential         | 4.8 M  | eval
1 | accuracy  | MulticlassAccuracy | 0      | eval
--------------------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.148    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [7]:
# Evaluate the model on the test set
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, num_workers=8, shuffle=False)
trainer.test(model=model, dataloaders=test_loader)

c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.3133758008480072     │
│         test_loss         │    1.9762518405914307     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.3133758008480072, 'test_loss': 1.9762518405914307}]

# Network 1: Basic CNN

In [ ]:
# Create basic CNN model using Immagenette dataset
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics


class CnnModel(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        # use cnn
        self.estimator = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.estimator(x)  # No need to reshape here!


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:

# prepare data for cnn taking 64x64 greyscale images
cnn_train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.Grayscale(),  # Move this before ToTensor()
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Adjust normalization for 1 channel
])

cnn_test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
cnn_train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=False, transform=cnn_train_transforms)

# Use 10% of the training set for validation
cnn_train_set_size = int(len(cnn_train_dataset) * 0.9)
cnn_val_set_size = len(cnn_train_dataset) - cnn_train_set_size

seed = torch.Generator().manual_seed(42)
cnn_train_dataset, cnn_val_dataset = torch.utils.data.random_split(cnn_train_dataset, [cnn_train_set_size, cnn_val_set_size], generator=seed)
cnn_val_dataset.dataset.transform = cnn_test_transforms

# Use DataLoader to load the dataset
cnn_train_loader = torch.utils.data.DataLoader(cnn_train_dataset, batch_size=128, num_workers=8, shuffle=True)
cnn_val_loader = torch.utils.data.DataLoader(cnn_val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
cnn_test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=False, transform=cnn_test_transforms)

cnn_model = CnnModel()



In [19]:
# Train cnn model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback], max_epochs=3)
trainer.fit(model=cnn_model, train_dataloaders=cnn_train_loader, val_dataloaders=cnn_val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | estimator | Sequential         | 2.1 M  | train
1 | accuracy  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.470     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [20]:
# Evaluate the model on the test set
trainer.test(model=cnn_model, dataloaders=test_loader)

c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.4188534915447235     │
│         test_loss         │    2.3453874588012695     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.4188534915447235, 'test_loss': 2.3453874588012695}]

# Network 2: ResNet 18

In [1]:
# Create a ResNet model using the Imagenette dataset
import torchvision.models as models
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics


class ResNetModel(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        # Use pretrained ResNet18
        self.estimator = models.resnet18(pretrained=True)
        
        # Modify the final layer to match the number of classes
        self.estimator.fc = nn.Linear(self.estimator.fc.in_features, num_classes)

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [6]:
from torchvision import transforms
from torchvision.datasets import Imagenette
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
# prepare data for ResNet 18 model taking 64x64 RGB images
resnet_train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

resnet_test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

resnet_train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=False, transform=resnet_train_transforms)

# Use 10% of the training set for validation
resnet_train_set_size = int(len(resnet_train_dataset) * 0.9)
resnet_val_set_size = len(resnet_train_dataset) - resnet_train_set_size

seed = torch.Generator().manual_seed(42)
resnet_train_dataset, resnet_val_dataset = torch.utils.data.random_split(resnet_train_dataset, [resnet_train_set_size, resnet_val_set_size], generator=seed)
resnet_val_dataset.dataset.transform = resnet_test_transforms

# Use DataLoader to load the dataset
resnet_train_loader = torch.utils.data.DataLoader(resnet_train_dataset, batch_size=128, num_workers=8, shuffle=True)
resnet_val_loader = torch.utils.data.DataLoader(resnet_val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
resnet_test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=False, transform=resnet_test_transforms)

resnet_model = ResNetModel()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

In [7]:
# Train ResNet model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback], max_epochs=3)
trainer.fit(model=resnet_model, train_dataloaders=resnet_train_loader, val_dataloaders=resnet_val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | estimator | ResNet             | 11.2 M | train
1 | accuracy  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\Users\baile\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [8]:
# Evaluate the model on the test set
trainer.test(model=resnet_model, dataloaders=test_loader)

NameError: name 'test_loader' is not defined

# CNN with Dropout

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics


class CnnModelWithDropout(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        # use cnn with dropout
        self.estimator = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),  # Add dropout layer
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),  # Add dropout layer
            
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Add dropout layer
            
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.estimator(x)  # No need to reshape here!


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
# prepare data for cnn with dropout taking 64x64 greyscale images
cnn_dropout_train_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.Grayscale(),  # Move this before ToTensor()
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Adjust normalization for 1 channel
])

cnn_dropout_test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
cnn_dropout_train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=False, transform=cnn_dropout_train_transforms)

# Use 10% of the training set for validation
cnn_dropout_train_set_size = int(len(cnn_dropout_train_dataset) * 0.9)
cnn_dropout_val_set_size = len(cnn_dropout_train_dataset) - cnn_dropout_train_set_size

cnn_dropout_train_dataset, cnn_dropout_val_dataset = torch.utils.data.random_split(cnn_dropout_train_dataset, [cnn_dropout_train_set_size, cnn_dropout_val_set_size], generator=seed)
cnn_dropout_val_dataset.dataset.transform = cnn_dropout_test_transforms

# Use DataLoader to load the dataset
cnn_dropout_train_loader = torch.utils.data.DataLoader(cnn_dropout_train_dataset, batch_size=128, num_workers=8, shuffle=True)
cnn_dropout_val_loader = torch.utils.data.DataLoader(cnn_dropout_val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
cnn_dropout_test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=False, transform=cnn_dropout_test_transforms)

cnn_dropout_model = CnnModelWithDropout()


# Transfer Learning on CIFAR10 Using Dropout CNN Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import CIFAR10
import pytorch_lightning as pl
import torchmetrics


class CnnModelWithDropout(pl.LightningModule):
    def __init__(self, data_path, batch_size=128, lr=1e-3):
        super().__init__()
        
        self.data_path = data_path
        self.batch_size = batch_size
        self.lr = lr
        
        # Load CIFAR-10 dataset
        dataset = CIFAR10(data_path, transform=transforms.Compose([
            transforms.Resize(64),  # Match CNN input size
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ]), download=True)
        
        dataset_size = len(dataset)
        train_size = int(dataset_size * 0.9)
        val_size = dataset_size - train_size
        
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
        
        # Define CNN model with dropout
        self.estimator = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(128, 10)
        )
        
        # Loss & Accuracy
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = self.accuracy(y_hat, y)
        
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_accuracy", acc, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=8)


# Train CNN with dropout on CIFAR-10
model_cnn_dropout = CnnModelWithDropout(data_path="./data")
